## Notebook Imports

In [170]:
import pandas as pd
import numpy as np

## Constants

In [171]:
VOCAB_SIZE = 2500

TRAINING_DATA_FILE = "SpamData/02_Training/train-data.txt"
TEST_DATA_FILE = "SpamData/02_Training/test-data.txt"

TOKEN_SPAM_PROB_FILE = "SpamData/03_Testing/prob-spam.txt"
TOKEN_HAM_PROB_FILE = "SpamData/03_Testing/prob-nonspam.txt"
TOKEN_ALL_PROB_FILE = "SpamData/03_Testing/prob-all-tokens.txt"

TEST_FEATURE_MATRIX = "SpamData/03_Testing/test-features.txt"
TEST_TARGET_FILE = "SpamData/03_Testing/test-target.txt"

# Read and Load Features from .txt Files into NumPy Array

In [172]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [173]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [174]:
print("Nr of rows in training file: ", sparse_train_data.shape[0])
print("Nr of rows in test file: ", sparse_test_data.shape[0])

Nr of rows in training file:  265427
Nr of rows in test file:  110522


In [175]:
print("Nr of emails in training file: ", np.unique(sparse_train_data[:, 0]).size)
print("Nr of emails in test file: ", np.unique(sparse_test_data[:, 0]).size)

Nr of emails in training file:  4015
Nr of emails in test file:  1724


# Create empty DataFrame

In [176]:
# Empty DF
column_names = ["DOC_ID"] + ["CATEGORY"] + list(range(VOCAB_SIZE))
index_names = np.unique(sparse_train_data[:, 0])

In [177]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

# Create a Full Matrix from a Sparse Matrix

In [178]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st columm
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurence of the word in sparse matrix. Default: 4th column
    """
    column_names = ["DOC_ID"] + ["CATEGORY"] + list(range(VOCAB_SIZE))
    doc_id_names = np.unique(sparse_train_data[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, "DOC_ID"] = doc_nr
        full_matrix.at[doc_nr, "CATEGORY"] = label
        full_matrix.at[doc_nr, word_id] = occurrence
        
        
    full_matrix.set_index("DOC_ID", inplace=True)
    return full_matrix

In [179]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 5.78 s, sys: 91.4 ms, total: 5.87 s
Wall time: 5.91 s


In [180]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,2,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,2,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,3,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,1,2,4,2,3,1,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [181]:
spam_probability = full_train_data.CATEGORY.sum() / full_train_data.shape[0]

In [182]:
spam_probability

0.31133250311332505

# Total Number of Words / Tokens

In [183]:
full_train_features = full_train_data.loc[:, full_train_data.columns != "CATEGORY"]

In [184]:
email_lengths = full_train_features.sum(axis=1)

In [185]:
total_wc = email_lengths.sum()

# Number of Tokens in Spam & Ham Emails

In [186]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_wc = spam_lengths.sum()
ham_wc = ham_lengths.sum()

In [187]:
print("Average number of words in spam email: ", spam_wc/spam_lengths.shape[0])
print("Average number of words in ham email: ", ham_wc/ham_lengths.shape[0])

Average number of words in spam email:  90.50488245931284
Average number of words in ham email:  156.516


# Summing the Tokens Occuring in Spam

In [188]:
full_train_features.shape

(4015, 2500)

In [189]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1

In [190]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [191]:
train_ham_tokens.shape

(2765, 2500)

## P(Token | Spam) - Probability that a Token Occurs given the Email is Spam

In [192]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)

## P(Token | Ham) - Probability that a Token Occurs given the Email is Ham

In [193]:
prob_tokens_ham = summed_ham_tokens / (ham_wc + VOCAB_SIZE)

# P(Token) - Probability that Token Occurs

In [194]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

# Save the Trained Model

In [195]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [196]:
sparse_test_data.shape

(110522, 4)

In [197]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

CPU times: user 18.8 s, sys: 657 ms, total: 19.5 s
Wall time: 19.9 s


In [198]:
X_test = full_test_data.loc[:, full_test_data.columns != "CATEGORY"]
y_test = full_test_data.CATEGORY

In [199]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)